In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [3]:
mnist_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5], std=[0.5]),
                                      transforms.Lambda(lambda x: x.view(-1, 784)),
                                      transforms.Lambda(lambda x: torch.squeeze(x))
                                      ])

data = datasets.MNIST(root='.', download=True, transform=mnist_transforms)

mnist_dataloader = DataLoader(data, batch_size=128, shuffle=True, num_workers=4) 

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
class Generator(nn.Module):
  '''
  Generator class in a CGAN. Accepts a noise tensor (latent dim 100)
  and a label tensor as input as outputs another tensor of size 784.
  Objective is to generate an output tensor that is indistinguishable 
  from the real MNIST digits.
  '''

  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(10, 10)
    self.layer1 = nn.Sequential(nn.Linear(in_features=100+10, out_features=256),
                                nn.LeakyReLU())
    self.layer2 = nn.Sequential(nn.Linear(in_features=256, out_features=512),
                                nn.LeakyReLU())
    self.layer3 = nn.Sequential(nn.Linear(in_features=512, out_features=1024),
                                nn.LeakyReLU())
    self.output = nn.Sequential(nn.Linear(in_features=1024, out_features=28*28),
                                nn.Tanh())

  def forward(self, z, y):
    # pass the labels into a embedding layer
    labels_embedding = self.embedding(y)
    # concat the embedded labels and the noise tensor
    # x is a tensor of size (batch_size, 110)
    x = torch.cat([z, labels_embedding], dim=-1)    
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.output(x)
    return x



In [ ]:

class Discriminator(nn.Module):
  '''
  Discriminator class in a CGAN. Accepts a tensor of size 784 and
  a label tensor as input and outputs a tensor of size 1,
  with the predicted class probabilities (generated or real data)
  '''

  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(10, 10)
    self.layer1 = nn.Sequential(nn.Linear(in_features=28*28+10, out_features=1024),
                                nn.LeakyReLU())
    self.layer2 = nn.Sequential(nn.Linear(in_features=1024, out_features=512),
                                nn.LeakyReLU())
    self.layer3 = nn.Sequential(nn.Linear(in_features=512, out_features=256),
                                nn.LeakyReLU())
    self.output = nn.Sequential(nn.Linear(in_features=256, out_features=1),
                                nn.Sigmoid())
    
  def forward(self, x, y):
    # pass the labels into a embedding layer
    labels_embedding = self.embedding(y)
    # concat the embedded labels and the input tensor
    # x is a tensor of size (batch_size, 794)
    x = torch.cat([x, labels_embedding], dim=-1)    
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.output(x)
    return x


In [ ]:
nz=100
device=torch.device("cuda" if(torch.cuda.is_available()) else "cpu")
G=Generator().device(device)
D=Discriminator().device(device)
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


$$
\min_G\max_D V(D,G)=\mathbb{E}_{x\sim p_{data}}[\log D(x)]+\mathbb{E}_{z\sim p_z}[\log(1-D(G(z)))]

$$

In [ ]:

for i, (x, labels) in enumerate(mnist_dataloader):
    # Sample random noise and labels
    z = torch.randn(x.shape[0], 100, device=device)
    y = torch.randint(0, 10, size=(x.shape[0],), device=device)
    # Generate images
    generated_imgs = G(z, y)
    # Calculate the discriminator output for fake images
    d_out_fake = D(generated_imgs, y)
    # Calculate the discriminator output for real images
    d_out_real = D(x, y)

    # discriminator loss  
    d_loss = criterion(d_out_fake, torch.zeros_like(d_out_fake))\
             +criterion(d_out_real, torch.ones_like(d_out_real)) 
    
    
